## 1. Setup & Imports

In [1]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 180)
sns.set_style("whitegrid")

## 2. Loading Data (with chunking for memory safety)

In [2]:
file_path = r"C:\Users\LENOVO LEGION\Desktop\Folders\AUK - MSc in AI\Fall 2025\Data Mining\Datasets\US_Accidents_March23.csv"

# Loading in chunks to avoid memory crashes
chunk_size = 100_000
chunks = pd.read_csv(file_path, chunksize=chunk_size, low_memory=False)

# Concatenating into one DataFrame
df = pd.concat(chunks, ignore_index=True)

print(f"Shape: {df.shape}")
df.head()

Shape: (7728394, 46)


,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,Right lane blocked due to accident on I-70 Eas...,I-70 E,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,Brice Rd,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day


## 3. Initial Audit

In [3]:
# Quick info
df.info(memory_usage="deep")

# Missing values overview
missing_summary = df.isna().mean().sort_values(ascending=False)
print("Missingness (top 15):")
print(missing_summary.head(15))
print("Number of missing values:", df.isna().sum().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)       

## 4. Converting Datetime Columns

In [4]:
for col in ["Start_Time", "End_Time", "Weather_Timestamp"]:
    df[col] = pd.to_datetime(df[col], errors="coerce")

df[["Start_Time", "End_Time", "Weather_Timestamp"]].head(10)

,Start_Time,End_Time,Weather_Timestamp
0,2016-02-08 05:46:00,2016-02-08 11:00:00,2016-02-08 05:58:00
1,2016-02-08 06:07:59,2016-02-08 06:37:59,2016-02-08 05:51:00
2,2016-02-08 06:49:27,2016-02-08 07:19:27,2016-02-08 06:56:00
3,2016-02-08 07:23:34,2016-02-08 07:53:34,2016-02-08 07:38:00
4,2016-02-08 07:39:07,2016-02-08 08:09:07,2016-02-08 07:53:00
5,2016-02-08 07:44:26,2016-02-08 08:14:26,2016-02-08 07:51:00
6,2016-02-08 07:59:35,2016-02-08 08:29:35,2016-02-08 07:56:00
7,2016-02-08 07:59:58,2016-02-08 08:29:58,2016-02-08 07:56:00
8,2016-02-08 08:00:40,2016-02-08 08:30:40,2016-02-08 07:58:00
9,2016-02-08 08:10:04,2016-02-08 08:40:04,2016-02-08 08:28:00


In [5]:
df[["Start_Time", "End_Time", "Weather_Timestamp"]].dtypes

Start_Time           datetime64[ns]
End_Time             datetime64[ns]
Weather_Timestamp    datetime64[ns]
dtype: object

## 5. Handling Duplicates

In [6]:
df = df.drop_duplicates()
print("Remaining rows:", len(df))

Remaining rows: 7728394


## 6. Missing Value Treatment

In [7]:
# Numerical columns → fill with mean
num_cols_mean = ["Temperature(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)", "Wind_Speed(mph)"]
for col in num_cols_mean:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mean())

# Categorical columns → fill with mode
cat_cols_mode = ["Street", "City", "Zipcode", "Timezone", "Airport_Code", "Wind_Direction", "Weather_Condition"]
for col in cat_cols_mode:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mode()[0])

# Twilight-related → fill with mode
twilight_cols = ["Sunrise_Sunset", "Civil_Twilight", "Nautical_Twilight", "Astronomical_Twilight"]
for col in twilight_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mode()[0])

# Description → replace with placeholder
df["Description"] = df["Description"].fillna("Unknown")

## 7. Dropping High-Missingness Columns

In [8]:
# Based on exploration: End_Lat, End_Lng, Wind_Chill(F), Precipitation(in)
drop_cols = ["End_Lat", "End_Lng", "Wind_Chill(F)", "Precipitation(in)"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

print("Remaining columns:", df.shape[1])

Remaining columns: 42


## 8. Range & Sanity Checks

In [9]:
# Removing negative or implausible values
df = df[df["Temperature(F)"] > -65]   # realistic lower bound
df = df[df["Temperature(F)"] < 135]   # realistic upper bound
df = df[df["Visibility(mi)"] >= 0]
df = df[df["Wind_Speed(mph)"] >= 0]
df = df[df["Pressure(in)"] > 0]

# Distance sanity
df = df[df["Distance(mi)"] >= 0]

## 9. Feature Engineering

In [10]:
# Accident duration in seconds
df["Duration_sec"] = (df["End_Time"] - df["Start_Time"]).dt.total_seconds()

# Remove extreme durations (> 7 days)
df = df[df["Duration_sec"] <= 7*24*3600]

# Extracting useful time features
df["Year"] = df["Start_Time"].dt.year
df["Month"] = df["Start_Time"].dt.month
df["Hour"] = df["Start_Time"].dt.hour
df["DayOfWeek"] = df["Start_Time"].dt.day_name()

## 10. Final Checks

In [11]:
print("Final shape:", df.shape)
print("Remaining missing values:", df.isna().sum().sum())

df.describe(include="all").T.head(20)

Final shape: (6979258, 47)
Remaining missing values: 106150


,count,unique,top,freq,mean,min,25%,50%,75%,max,std
ID,6979258,6979258,A-1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Source,6979258,3,Source1,3576594,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Severity,6979258.0,NaN,NaN,NaN,2.229127,1.0,2.0,2.0,2.0,4.0,0.498562
Start_Time,6979258,NaN,NaN,NaN,2020-03-21 15:34:02.347961856,2016-02-08 00:37:08,2018-09-15 21:46:54.750000128,2020-06-22 22:12:30,2021-10-28 15:46:22.249999872,2023-03-31 23:30:00,NaN
End_Time,6979258,NaN,NaN,NaN,2020-03-21 17:26:45.718612992,2016-02-08 06:37:08,2018-09-15 23:01:06.249999872,2020-06-23 00:00:00,2021-10-28 17:27:58.750000128,2023-03-31 23:59:00,NaN
Start_Lat,6979258.0,NaN,NaN,NaN,36.22762,24.5548,33.42626,35.815583,40.096209,49.002201,5.05445
Start_Lng,6979258.0,NaN,NaN,NaN,-94.765089,-124.623833,-117.218234,-87.88539,-80.386212,-67.113167,17.358294
Distance(mi),6979258.0,NaN,NaN,NaN,0.515641,0.0,0.0,0.01,0.387,441.75,1.742222
Description,6979258,3541204,A crash has occurred causing no to minimum del...,7871,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Street,6979258,319536,I-95 N,79767,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 11. Saving the Cleaned Dataset

In [15]:
df.to_csv(r"C:\Users\LENOVO LEGION\Desktop\Folders\AUK - MSc in AI\Fall 2025\Data Mining\Datasets\US_Accidents_Cleaned.csv", index=False)
print("Cleaned dataset saved.")

Cleaned dataset saved.
